In [ ]:
from qubic import fibtools as ft
import fitting as fit
from importlib import reload

rc('figure',figsize=(10,6))
rc('font',size=12)


In [ ]:
######### iminuit direct
x = np.linspace(0,1,10)
sig = np.ones(len(x)) * 1.
errors = np.random.randn(len(x)) * sig
y = 3*x+1 + errors

def linearfit(x, pars):
    return pars[0] + pars[1] * x

import iminuit
from iminuit.cost import LeastSquares
myminimizer = LeastSquares(x, y, errors, linearfit)
m = iminuit.Minuit(myminimizer, [0., 0.], name=None)
m.migrad()
m.hesse()

print(m)

errorbar(x, y, yerr=sig, fmt='ko')
plot(x, linearfit(x, m.values), 'r', lw=2)

In [ ]:
m.interactive()

In [ ]:
### Use the fitting.py library 

reload(fit)

x = np.linspace(0,1,10)
sig = np.ones(len(x)) * 1.
errors = np.random.randn(len(x)) * sig
y = 3*x+1 + errors

def linearfit(x, pars):
    return pars[0] + pars[1] * x

mydata = fit.Data(x,y,sig, linearfit, pnames=['a', 'b'])

guess = [0., 0.]
mydata.fit_minuit(guess)
mydata.plot()

In [ ]:
m.interactive()

# 2D fitting

In [ ]:
x = np.linspace(-10,10,101)
y = x.copy()
xx,yy = meshgrid(x,y)

signoise = 0.3
mapxy = 3. * np.exp(-0.5 * ((xx-2)**2 + (yy+3)**2)/0.5) + np.random.randn(*np.shape(xx))*signoise

imshow(mapxy, origin='lower', extent=[np.min(x), np.max(x), np.min(y), np.max(y)])
xlabel('Arcminutes')
ylabel('Arcminutes')
colorbar()


In [ ]:
class gauss2dfit:
    def __init__(self, xx, yy):
        self.xx = xx
        self.yy = yy
    def __call__(self, x, pars):
        amp, xc, yc, sig = pars
        mygauss = amp * np.exp(-0.5*((self.xx-xc)**2+(self.yy-yc)**2)/sig**2)
        return np.ravel(mygauss)
        

guess = np.array([np.max(mapxy), 0.,0., 1.])

g2d = gauss2dfit(xx, yy)


mm, ss = ft.meancut(mapxy, 3)

data = fit.Data(np.ravel(xx), np.ravel(mapxy), np.ravel(xx)*0+ss, g2d)
m, ch2, ndf = data.fit_minuit(guess)

subplot(2,3,1)
imshow(mapxy, origin='lower', extent=[np.min(x), np.max(x), np.min(y), np.max(y)])
xlabel('Arcminutes')
ylabel('Arcminutes')
colorbar()

fitted = np.reshape(g2d(x, m.values), (len(x), len(x)))

subplot(2,3,2)
imshow(fitted, origin='lower', extent=[np.min(x), np.max(x), np.min(y), np.max(y)])
xlabel('Arcminutes')
ylabel('Arcminutes')
colorbar()

subplot(2,3,3)
imshow(mapxy-fitted, origin='lower', extent=[np.min(x), np.max(x), np.min(y), np.max(y)])
xlabel('Arcminutes')
ylabel('Arcminutes')
colorbar()

tight_layout()

# Now using a user-defined cost function

In [ ]:
######### iminuit direct
x = np.linspace(0,1,10)
sig = np.ones(len(x)) * 1.
errors = np.random.randn(len(x)) * sig
y = 3*x+1 + errors

def linearfit(x, pars):
    return pars[0] + pars[1] * x

class mychi2:
    """
    Generic least-squares cost function with error.
    """
    
    errordef = iminuit.Minuit.LEAST_SQUARES # for Minuit to compute errors correctly

    def __init__(self, x, y, err, model):
        self.model = model  # model predicts y for given x
        self.x = np.asarray(x)
        self.y = np.asarray(y)
        self.err = np.asarray(err)

    def __call__(self, *par):  # we accept a variable number of model parameters
        ym = self.model(self.x, *par)
        return np.sum((self.y - ym) ** 2 / self.err ** 2)

    @property
    def ndata(self):
        return len(self.x)

import iminuit
from iminuit.cost import LeastSquares
myminimizer = mychi2(x, y, errors, linearfit)
m = iminuit.Minuit(myminimizer, [0., 0.], name=None)
m.migrad()
m.hesse()

print(m)

errorbar(x, y, yerr=sig, fmt='ko')
plot(x, linearfit(x, m.values), 'r', lw=2)

# Now with the fitting library

In [ ]:
### Use the fitting.py library 

reload(fit)

x = np.linspace(0,1,10)
sig = np.ones(len(x)) * 1.
errors = np.random.randn(len(x)) * sig
y = 3*x+1 + errors

def linearfit(x, pars):
    return pars[0] + pars[1] * x

class mychi2:
    """
    Generic least-squares cost function with error.
    """
    
    errordef = iminuit.Minuit.LEAST_SQUARES # for Minuit to compute errors correctly

    def __init__(self, x, y, err, model):
        self.model = model  # model predicts y for given x
        self.x = np.asarray(x)
        self.y = np.asarray(y)
        self.err = np.asarray(err)

    def __call__(self, *par):  # we accept a variable number of model parameters
        ym = self.model(self.x, *par)
        return np.sum((self.y - ym) ** 2 / self.err ** 2)

    @property
    def ndata(self):
        return len(self.x)

mydata = fit.Data(x,y,sig, linearfit, pnames=['a', 'b'])

guess = [0., 0.]
fitted, ch2, ndf = mydata.fit_minuit(guess, minimizer=mychi2)

mydata.plot()

In [ ]:
print(fitted.values)
print(np.array(fitted.values))

In [ ]:
print(fitted.errors)
print(np.array(fitted.errors))

In [ ]:
print(fitted.covariance)
print(np.array(fitted.covariance))

In [ ]:
vals = np.random.randn(1000)*2.5+6

In [ ]:
fit.myhist(vals, bins=10, unbinned=False)